# Tarea 3 - Pregunta 3 - Calidad de un vino
---
# Matías Araya - 201173082-8
# Claudia Hazard - 201404523-9

# a)
Carge los dos dataset en un único dataframe de pandas, además de agregar una columna indicando si
es vino tinto o blanco. Describa el dataset a trabajar.

In [ ]:
import pandas as pd
df_red = pd.read_csv("winequality-red.csv",sep=";")
df_white = pd.read_csv("winequality-white.csv",sep=";")
df = pd.concat([df_red,df_white], axis=0)
#genere atributo 'tipo

# b)
 Aborde este problema como si fuera de clasificación binaria para predecir si un vino es de buena calidad
o no, es decir, utilice las distintas características fisioquímicas presentes en los datos para estimar esta
etiqueta. Para esto cree las matrices de entrenamiento y de pruebas, además de la etiqueta para ambos
conjuntos, considerando como quality mayor a 5 un vino de buena calidad. El conjunto de pruebas
(25 %) será utilizado únicamente para verificar la calidad de los algoritmos a entrenar.

In [ ]:
df['good_quality'] = [1 if q>5 else 0 for q in df.quality] #then remove 'quality' from df
2 #train and test split over df

# c)
Entrene un solo Arbol de Clasificación de múltiples niveles para resolver el problema. Puede variar los ´
hiper-parámetros que prefiera, recuerde que las decisiones no pueden ser basadas mirando el conjunto
de pruebas. Debido al desbalanceo que se produce en las dos clases mida la m´etrica F1-score [5] sobre
el conjunto de entrenamiento y de pruebas.

# d)
Entrene un ensamblador de árboles de múltiples niveles, mediante la técnica de Random Forest. Varíe la
cantidad de árboles de decisión utilizados en el ensamblado (n estimators), realice un gráfico resumen
del F1-score de entrenamiento y de pruebas en función de este hiper-parámetro.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=, max_depth=,n_jobs=-1)
from sklearn.metrics import f1_score
f1_score(y_true, y_pred)

# e)
 Entrene un ensamblador de árboles de múltiples niveles, mediante la técnica de AdaBoost. Varíe la
cantidad de árboles de decisión utilizados en el ensamblado (n estimators), realice un gráfico resumen
del F1-score de entrenamiento y de pruebas en función de este hiper-parámetro. Compare y analice con
la técnica utilizada en d).

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(base_estimator=Tree(max_depth=), n_estimators=)

# f)
Entrene alguna otra máquina de aprendizaje, elegida por usted, para resolver este problema. Elija los
hiper-parámetros que estime convenientes intentando aumentar el F1-score obtenido por los algoritmos
anteriores. Compare y analice estas 4 maneras de resolver el problema definido en b).

# g)
Defina un criterio para estimar la importancia de los distintos atributos en el ensamblado de Random Forest, implementelo sobre alguno de los ensambladores entrenados en d), haga un ranking de importancia de atributos >Es posible implementar este criterio sobre una técnica de boost como lo es AdaBoost?